In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path

In [ ]:
bugs = pd.read_csv('../data/generated/buggy-code-data.csv', sep=',')

In [ ]:
bugs.head()

In [ ]:
bugs['Authors'] = ''
bugs['Number of Revisions'] = ''
bugs['Dates'] = ''

In [ ]:
length = bugs.shape[0]
j = 0

for i in range(length):
    
    sha1 = (bugs['fix_commit_hash'].iloc[i])
    bug_id = (bugs['bug_id'].iloc[i])
    bug_type = bugs['bug_type'].iloc[i]
    sub_repo_url = bugs['project_full_name'].iloc[i].split('/')[1]
    bug_line = bugs['buggy_line_number'].iloc[i]
    filename = bugs['buggy_file_path'].iloc[i].split('/')[-1]
    path = bugs['buggy_file_path'].iloc[i].rsplit('/', 1)[0]
    
    print(filename)
    print(path)

    process = subprocess.Popen(['git','checkout',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process.communicate()
    
    try:
        process2 = subprocess.Popen(['git','log','--date=unix','-L', str(bug_line) + ',' + str(bug_line) + ':' + str(filename)],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url) + '\\' + path, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
        output2, errors = process2.communicate()

        commits = []
        authors = []
        dates = []

        for linha in output2.splitlines():

            #print(linha)

            if linha.startswith('commit'):

                commit = linha.split("commit")[1].strip()
                #print(commit)
                commits.append(commit)

            if linha.startswith('Author:'):

                author = linha.split("Author:")[1].strip().split('<',1)[0]
                #print(author)
                authors.append(author)

            if linha.startswith('Date:'):

                date = linha.split("Date:")[1].strip()
                dates.append(date)

        bugs.at[i,'Authors'] = authors
        bugs.at[i,'Number of Revisions'] = commits
        bugs.at[i,'Dates'] = dates

    except:
        
        print('Invalid Directory')

In [ ]:
bugs['Fix commits'] = ''

In [ ]:
import re
length = bugs.shape[0]

for i in range(length):
    
    commits = (bugs.loc[i,['Number of Revisions']])
    fix_commits = []
    for commit in commits:
        
        if not commit:
            print("List is empty")
            
        else:
            
            ## search for message
            
            process = subprocess.Popen(['git','log','--format=%B','-n','1',str(commit[0])],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
            output, errors = process.communicate()
            
            for output_line in output.splitlines():
                
                pattern = re.search("fix(e[ds])?|bugs?|defects?|patch|corrigidos?|close([sd])?|resolve([sd])?", output_line, re.I)
                
                if pattern is not None:
                
                    #print(pattern)
                    fix_commits.append(str(commit[0]))
                    break

    bugs.at[i,'Fix commits'] = fix_commits
                

In [ ]:
bugs['Bug Probability'] = ''

In [ ]:
import numpy as np
import math

def date_normalized(x,y):
    
    dates = []
    for commit in x:
        
        process = subprocess.Popen(['git','show','--no-patch','--no-notes','--pretty=' + r'%cd','--format=' + r"%at",commit],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + y, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
        output, errors = process.communicate()
        
        print(str(output.splitlines()[0].rstrip()))
        dates.append(int(output.splitlines()[0].rstrip()))
       
        
    return dates



def NormalizeData(data):
    
    data =  np.array(data)
    
    if len(data) == 1:
        
        return [1]
    
    else:
        
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    


def TWRFunction(normal_data):
    
    twr = 0
    for i in normal_data:
        
        twr = twr + 1/1 + math.e**(-12*i + 12)
    
    defect_probability = 1 -math.e**-twr
    
    return defect_probability
        

In [ ]:
length = bugs.shape[0]

for i in range(length):
    
    #print(buggy_elements_df['Commits'].iloc[i])
    #print(buggy_elements_df['Repo'].iloc[i])
    
    if not bugs['Fix commits'].iloc[i]:
        
        try:
            bugs.at[i,'Bug Probability'] = 0.3 * TWRFunction(NormalizeData(date_normalized(bugs['Number of Revisions'].iloc[i],bugs['project_full_name'].iloc[i].split('/')[1]))) 
        
        except ValueError:
             print('Commit List is Empty')
            
    else:    
    
        try:
            bugs.at[i,'Bug Probability'] = 0.7 * TWRFunction(NormalizeData(date_normalized(bugs['Fix commits'].iloc[i],bugs['project_full_name'].iloc[i].split('/')[1]))) + 0.3 * TWRFunction(NormalizeData(date_normalized(bugs['Number of Revisions'].iloc[i],bugs['project_full_name'].iloc[i].split('/')[1])))

        except ValueError:
            print('Commit List is Empty')

In [ ]:
bugs['Bug Probability'].unique()

In [ ]:
#bugs.to_csv('..\data\RQ3_Data.csv', index=False, encoding="utf8" )